In [1]:
#! python2
# -*- coding: utf-8 -*-
import pprint
import pymongo
import datetime
import numpy as np
from pymongo import MongoClient

from scipy import spatial

import pylab

import os

import progressbar

db_name = 'twitter'
col_name = 'after_process_replab'

In [2]:
from sklearn import metrics
from sklearn.cluster import DBSCAN
# from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

from igraph import *
import igraph
import math
from operator import itemgetter



In [3]:
class MyPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, unicode):
            return (object.encode('thai'), True, False)
        return pprint.PrettyPrinter.format(self, object, context, maxlevels, level)

def go_print( input ):
    MyPrettyPrinter().pprint(input)
    # ppp = pprint.PrettyPrinter(indent=4)
    # ppp.pprint(input)
    return;

def get_midnight(time):
    return time.replace(minute=0, hour=0, second=0, microsecond=0)

def get_time_gap(time,hour_gap=1,min_gap=1):
    h = time.hour
    m = time.minute
    o_h = h/hour_gap*hour_gap
    o_m = m/min_gap*min_gap
    return time.replace(hour=o_h, minute=o_m, second=0, microsecond=0)

def get_week_year(time):
    return tuple([time.isocalendar()[0], time.isocalendar()[1]])

def get_thai_midnight(time):
    out = time + datetime.timedelta(hours=7)
    out = out.replace(minute=0, hour=0, second=0, microsecond=0) - datetime.timedelta(hours=7)
    return out

In [4]:
f = open("replab2013_entities.tsv","r") #opens file with name of "test.txt"
class_set = {}
for line in f:
    x = line.split('\t')
#     class_set.append( line)
    class_set[ x[0].strip('"') ] = [x[1].strip('"'),x[3].strip('"')] 
print class_set

{'entity_id': ['query', 'category'], 'RL2013D04E153': ['Bon Jovi', 'music'], 'RL2013D04E149': ['beatles', 'music'], 'RL2013D04E207': ['Britney Spears', 'music'], 'RL2013D04E169': ['Jennifer Lopez', 'music'], 'RL2013D04E164': ['Coldplay', 'music'], 'RL2013D03E093': ['Yale University', 'university'], 'RL2013D03E090': ['Princeton University', 'university'], 'RL2013D03E091': ['Columbia University', 'university'], 'RL2013D03E096': ['Johns Hopkins University', 'university'], 'RL2013D03E097': ['New York University', 'university'], 'RL2013D04E145': ['Adele', 'music'], 'RL2013D04E162': ['Maroon 5', 'music'], 'RL2013D02E063': ['Capital One', 'banking'], 'RL2013D02E060': ['PNC', 'banking'], 'RL2013D04E166': ['Lady Gaga', 'music'], 'RL2013D02E067': ['Banco Santander', 'banking'], 'RL2013D04E161': ['The wanted', 'music'], 'RL2013D02E068': ['Bankia', 'banking'], 'RL2013D04E185': ['Shakira', 'music'], 'RL2013D04E146': ['Alicia Keys', 'music'], 'RL2013D01E005': ['Toyota', 'automotive'], 'RL2013D01E003

In [11]:
if __name__ == '__main__':
    client = MongoClient()
    db = client[db_name]

    result = db[col_name].create_index([('ts', pymongo.ASCENDING)])
    cursor = db[col_name].find({})

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    topic_per_time = {}
    for doc in cursor:
        data += 1
        bar.update(data)
        ts = doc['ts']
        entity_id = doc['entity_id']
        topic = doc['topic']
       
        minor_class = class_set[entity_id][0]
        
        major_class = class_set[entity_id][1]
        
        db[col_name].update_one({'_id':doc['_id']},{"$set":{'major_class':major_class,'minor_class':minor_class}}, upsert=True)
        
        
        datetime_object = datetime.datetime.fromtimestamp(ts)
        time_gap = get_midnight(datetime_object)
        
        if not topic_per_time.has_key(time_gap):
            topic_per_time[time_gap] = {}
        if not topic_per_time[time_gap].has_key(topic):
             topic_per_time[time_gap][topic] = 0
        topic_per_time[time_gap][topic] += 1
          
    bar.finish()
    
    ignore_topics = ['picture','link']
    
    cursor = db[col_name].find({})

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    for doc in cursor:
        data += 1
        bar.update(data)
        ts = doc['ts']
        entity_id = doc['entity_id']
        topic = doc['topic']
        
        datetime_object = datetime.datetime.fromtimestamp(ts)
        time_gap = get_midnight(datetime_object)
        
        topic_dense = 0
        topic_l = topic.lower()
        check = True
        if topic =='':
            check = False
        for ignore in ignore_topics:
            if ignore in topic_l:
                check = False
        if topic_per_time[time_gap][topic] >= 3 and check:
            topic_dense = 1
        db[col_name].update_one({'_id':doc['_id']},{"$set":{'topic_dense':topic_dense}}, upsert=True)
    bar.finish()
#     print len(date)

[########################################################################] 100%
[########################################################################] 100%


In [8]:
ignore_topic = ['','picture','link']
ignore_topic[2] in 'Web links to Britney'

True

In [ ]:
str = "0000000this is string example....wow!!!0000000";
print str.strip( '0' )

In [ ]:
# class_set = [
#     ["RL2013D01E001","BMW","automotive"],
#     ["RL2013D01E002","Audi","automotive"],
#     ["RL2013D01E003","Volvo","automotive"],
#     ["RL2013D01E005","Toyota","automotive"],
#     ["RL2013D01E008","Volkswagen","automotive"],
#     ["RL2013D01E009","Honda","automotive"],
#     ["RL2013D01E012","Nissan","automotive"],
#     ["RL2013D01E013","Fiat","automotive"],
#     ["RL2013D01E014","Suzuki","automotive"],
#     ["RL2013D01E015","Mazda","automotive"],
#     ["RL2013D01E016","Chrysler","automotive"],
#     ["RL2013D01E019","Subaru","automotive"],
#     ["RL2013D01E022","Ferrari","automotive"],
#     ["RL2013D01E025","Bentley","automotive"],
#     ["RL2013D01E033","Porsche","automotive"],
#     ["RL2013D01E035","Yamaha","automotive"],
#     ["RL2013D01E040","Kia","automotive"],
#     ["RL2013D01E041","Ford","automotive"],
#     ["RL2013D01E043","Jaguar","automotive"],
#     ["RL2013D01E044","Lexus","automotive"],
#     ["RL2013D02E051","RBS bank","banking"],
#     ["RL2013D02E054","Barclays","banking"],
#     ["RL2013D02E055","HSBC","banking"],
#     ["RL2013D02E056","Bank of America","banking"],
#     ["RL2013D02E057","Wells Fargo","banking"]
#     ["RL2013D02E060","PNC","banking"],
#     ["RL2013D02E063","""Capital One""","banking"],
#     ["RL2013D02E067","Banco Santander","banking"],
#     ["RL2013D02E068","Bankia","banking"],
#     ["RL2013D02E070","BBVA","banking"],
#     ["RL2013D02E076","Goldman Sachs","banking"],
#     ["RL2013D03E086","Harvard University","university"],
#     ["RL2013D03E087","Stanford University","university"],
#     ["RL2013D03E088","Berkeley University","university"],
#     ["RL2013D03E089","MIT","university"],
#     ["RL2013D03E090","Princeton University","university"],
#     ["RL2013D03E091","Columbia University","university"],
#     ["RL2013D03E093","Yale University","university"],
#     ["RL2013D03E096","Johns Hopkins University","university"],
#     ["RL2013D03E097","New York University","university"],
#     ["RL2013D03E124","Oxford University","university"],
#     ["RL2013D04E145","Adele","music"],
#     ["RL2013D04E146","Alicia Keys","music"],
#     ["RL2013D04E149","beatles","music"],
#     ["RL2013D04E151","Led Zeppelin","music"],
#     ["RL2013D04E152","Aerosmith","music"],
#     ["RL2013D04E153","Bon Jovi","music"],
#     ["RL2013D04E155","U2","music"],
#     ["RL2013D04E159","AC/DC","music"],
#     ["RL2013D04E161","The wanted","music"],
#     ["RL2013D04E162","Maroon 5","music"],
#     ["RL2013D04E164","Coldplay","music"],
#     ["RL2013D04E166","Lady Gaga","music"],
#     ["RL2013D04E167","madonna","music"],
#     ["RL2013D04E169","Jennifer Lopez","music"],
#     ["RL2013D04E175","Justin Bieber","music"],
#     ["RL2013D04E185","Shakira","music"],
#     ["RL2013D04E194","PSY","music"],
#     ["RL2013D04E198","The Script","music"],
#     ["RL2013D04E206","Whitney Houston","music"],
#     ["RL2013D04E207","Britney Spears","music"]
# ]